### Dependencies

In [18]:
import pandas as pd
from pathlib import Path
import re, ast, numpy as np

### 1. Data Exploration/Preparation

In [8]:
df = pd.read_csv(Path("../data/external/kaggle_spotify_data.csv"))

In [9]:
df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [10]:
df.dtypes

Unnamed: 0            int64
track_id             object
artists              object
album_name           object
track_name           object
popularity            int64
duration_ms           int64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
track_genre          object
dtype: object

In [11]:
df = df.drop(columns=["Unnamed: 0"])

Removed column because it just repeated the index of the rows.

In [17]:
df.duplicated().sum()
dupes = df[df.duplicated(keep=False)]
dupes.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
1874,0CDucx9lKxuCZplLXUz0iX,Buena Onda Reggae Club,Disco 2,Song for Rollins,16,219346,False,0.841,0.577,0,-7.544,1,0.0438,0.238000,0.860000,0.0571,0.843,90.522,4,afrobeat
1925,0CDucx9lKxuCZplLXUz0iX,Buena Onda Reggae Club,Disco 2,Song for Rollins,16,219346,False,0.841,0.577,0,-7.544,1,0.0438,0.238000,0.860000,0.0571,0.843,90.522,4,afrobeat
2109,2aibwv5hGXSgw7Yru8IYTO,Red Hot Chili Peppers,Stadium Arcadium,Snow (Hey Oh),80,334666,False,0.427,0.900,11,-3.674,1,0.0499,0.116000,0.000017,0.1190,0.599,104.655,4,alt-rock
2155,2aibwv5hGXSgw7Yru8IYTO,Red Hot Chili Peppers,Stadium Arcadium,Snow (Hey Oh),80,334666,False,0.427,0.900,11,-3.674,1,0.0499,0.116000,0.000017,0.1190,0.599,104.655,4,alt-rock
3683,7mULVp0DJrI2Nd6GesLvxn,Joy Division,Timeless Rock Hits,Love Will Tear Us Apart,0,204621,False,0.524,0.902,2,-8.662,1,0.0368,0.000989,0.695000,0.1370,0.907,146.833,4,alternative


In [22]:
df["track_name_clean"] = (df['track_name'].fillna("").astype(str).str.lower().str.replace(r"\s*\([^)]*\)\s*$", "", regex=True).str.replace(r"\s+", " ", regex=True).str.strip())

"Song (Live)" and "Song (Remastered 2012)" become just "song"

In [39]:
multi_artist_songs = df[df['artists'].str.contains(';', na=False)]
len(multi_artist_songs)
multi_artist_songs[['artists', 'track_name']].head(10)

,artists,track_name
2,Ingrid Michaelson;ZAYN,To Begin Again
6,A Great Big World;Christina Aguilera,Say Something
8,Jason Mraz;Colbie Caillat,Lucky
14,Chord Overstreet;Deepend,Hold On - Remix
16,Andrew Foy;Renee Foy,ily (i love you baby)
17,Andrew Foy;Renee Foy,At My Worst
18,Jason Mraz;Colbie Caillat,Lucky
19,Boyce Avenue;Bea Miller,Photograph
21,Boyce Avenue;Jennel Garcia,Demons
22,A Great Big World;Christina Aguilera,Say Something


In [58]:
test_multi = df[df['track_name'].str.contains(r'feat\.|ft\.|featuring', case=False, na=False) & df['track_name'].str.contains(', ', na=False)].head(10)
test_multi


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,track_name_clean
1548,1elI9IOBskmT6IHYbtjc8F,Calle 13;Julian Assange;Tom Morello;Kamilya Ju...,"Multi-Viral (feat. Julian Assange, Tom Morello...","Multi-Viral (feat. Julian Assange, Tom Morello...",21,260877,False,0.501,0.963,6,...,1,0.2650,0.051600,0.000000,0.3070,0.656,165.128,4,afrobeat,multi-viral
1560,1xuYajTJZh8zZrPRmUaagf,Calle 13;Totó La Momposina;Susana Baca;Maria Rita,Entren Los Que Quieran,"Latinoamérica (feat. Totó la Momposina, Susana...",64,301426,False,0.691,0.805,2,...,0,0.0947,0.590000,0.000000,0.0975,0.866,142.503,3,afrobeat,latinoamérica
5838,0fp5eToibSlsPu10KsnyoT,Rustage;NerdOut;Lex Bratcher;Shao Dow,"Turn it Up (Team 7 Rap) [feat. NerdOut, Lex Br...","Turn it Up (Team 7 Rap) [feat. NerdOut, Lex Br...",40,215135,False,0.775,0.807,8,...,1,0.1310,0.145000,0.000000,0.1500,0.202,157.099,4,anime,"turn it up (team 7 rap) [feat. nerdout, lex br..."
7477,4gRQ6p0okTwWXLEiMg35e3,Dan Tyminski;Jerry Douglas;Todd Phillips;Sam B...,One More Time Before You Go,One More Time Before You Go (feat. Jerry Dougl...,23,246200,False,0.627,0.655,1,...,0,0.0307,0.488000,0.000002,0.2850,0.695,96.933,3,bluegrass,one more time before you go (feat. jerry dougl...
7544,4SiS7LvF6n8WnW5ZeB4RfL,The Seldom Scene;Emmylou Harris;Tom Gray;John ...,Long Time... Seldom Scene,"With Body and Soul (feat. Emmylou Harris, Tom ...",22,201573,False,0.511,0.269,4,...,1,0.0346,0.899000,0.000183,0.0897,0.435,139.793,4,bluegrass,with body and soul
10565,0MF391cptMRWJwkIynqf5V,Bassnectar;Jantsen;Fashawn;Zion I,Noise vs Beauty,"Lost in the Crowd (feat. Fashawn, Zion I)",38,212571,False,0.659,0.941,1,...,0,0.0434,0.000387,0.000000,0.4150,0.861,140.018,4,breakbeat,lost in the crowd
15552,5IzjE4F631lSstz8Co5U4O,GEMINI;CAMO;WOODZ,MIA,"MIA (Feat. CAMO, WOODZ)",55,195026,False,0.534,0.685,5,...,0,0.0681,0.202000,0.000000,0.0961,0.445,84.912,4,chill,mia
17347,4kRROniJRDFFhgzGv2pnD1,Glee Cast;Darren Criss,Glee: The Music presents The Warblers,"Hey, Soul Sister (feat. Darren Criss)",45,220546,False,0.664,0.533,2,...,1,0.0279,0.587000,0.000000,0.2250,0.628,97.032,4,club,"hey, soul sister"
17376,67QoEc0mR2OpqNKXgHGHDq,Glee Cast;Darren Criss,Glee: The Music presents The Warblers,"Bills, Bills, Bills (feat. Darren Criss)",46,180280,False,0.833,0.441,11,...,0,0.1520,0.392000,0.000000,0.1490,0.762,127.658,4,club,"bills, bills, bills"
17517,7irY7Ldcf6BFpycGl62vum,Glee Cast;Darren Criss,"Glee: The Music, The Christmas Album","Baby, It's Cold Outside (feat. Darren Criss)",49,166546,False,0.520,0.270,8,...,1,0.0299,0.616000,0.000000,0.1090,0.509,110.789,4,club,"baby, it's cold outside"


In [148]:
feat_songs = df[df['track_name'].str.contains(r'feat\.?\s|\bft\.?\s|featuring\.?', case=False, na=False)]
len(feat_songs)

2650

In [169]:
df['primary_artist(s)'] = [[] for _ in range(len(df))]
df['featured_artist(s)'] = [[] for _ in range(len(df))]


for idx, row in feat_songs.iterrows():
    track_name = row['track_name']
    original_artists = row['artists']

    primary_artists = []
    featured_artists = []

    match = re.search(r'(?:feat\.?\s|\bft\.?\s|featuring\.?)\s*([^)]+)', track_name, re.IGNORECASE)

    if match:
        feature_text = match.group(1).strip()

        if ',' in feature_text:
            multiple_features = [f.strip() for f in feature_text.split(',')]

            final_features = []
            for feature_group in multiple_features:
                if '&' in feature_group:
                    ampersand_split = [f.strip() for f in feature_group.split('&')]
                    final_features.extend(ampersand_split)
                else:
                    final_features.append(feature_group)
            
            featured_artists = final_features
        else:
            if '&' in feature_text:
                featured_artists = [f.strip() for f in feature_text.split('&')]
            else:
                featured_artists = [feature_text]

        if pd.notna(original_artists):
            primary_artists = [f.strip() for f in original_artists.split(';')]
            primary_artists = [artist for artist in primary_artists if artist not in featured_artists]

    else:
        if pd.notna(original_artists):
            primary_artists = [f.strip() for f in original_artists.split(';')]
        
    df.at[idx, 'primary_artist(s)'] = primary_artists
    df.at[idx, 'featured_artist(s)'] = featured_artists

In [167]:
no_feat_songs = df[~df['track_name'].str.contains(r'feat\.?\s|\bft\.?\s|featuring\.?', case=False, na=False)]
len(no_feat_songs)

111350

In [173]:
for idx, row in no_feat_songs.iterrows():
    track_name = row['track_name']
    original_artists = row['artists']

    primary_artists = []

    if pd.notna(original_artists):
        primary_artists = [artist.strip() for artist in original_artists.split(';')]

    df.at[idx, 'primary_artist(s)'] = primary_artists

In [174]:
df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,track_name_clean,primary_artist(s),featured_artist(s)
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,...,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,comedy,[Gen Hoshino],[]
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,...,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,ghost - acoustic,[Ben Woodward],[]
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,...,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,to begin again,"[Ingrid Michaelson, ZAYN]",[]
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,...,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,can't help falling in love,[Kina Grannis],[]
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,...,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,hold on,[Chord Overstreet],[]
